# Оптимизация расписания автобусов и водителей


Этот ноутбук предназначен для составления расписания водителей и автобусов на основе заданных условий:
- Автобусы работают с 6:00 до 03:00.
- Учитываются пиковые часы (7:00–9:00 и 17:00–19:00).
- Водители делятся на два типа: A (8-часовые смены) и B (24-часовые смены).
- Цель: минимизировать количество водителей и обеспечить максимальную работу автобусов.


In [ ]:
import pandas as pd
from datetime import timedelta, datetime

In [4]:

# Шаг 1. Настройка основных параметров задачи
КОЛИЧЕСТВО_АВТОБУСОВ = 10  # Общее количество автобусов
ВРЕМЯ_МАРШРУТА = timedelta(hours=1, minutes=15)  # Время маршрута
РАБОЧИЕ_ЧАСЫ = 21  # Часы работы (6:00 до 3:00)
ПИКОВЫЕ_ЧАСЫ_УТРОМ = [(7, 9)]  # Утренние часы пик
ПИКОВЫЕ_ЧАСЫ_ВЕЧЕРОМ = [(17, 19)]  # Вечерние часы пик
СМЕНА_A = 8  # Количество часов работы водителей типа А
СМЕНА_B = 24  # Количество часов работы водителей типа B

# Генерация времени интервалов
НАЧАЛО = datetime.strptime("06:00", "%H:%M")
КОНЕЦ = datetime.strptime("03:00", "%H:%M") + timedelta(days=1)

# Расчет интервалов
временные_интервалы = pd.date_range(start=НАЧАЛО, end=КОНЕЦ, freq="15min").to_pydatetime().tolist()


In [5]:

# Шаг 2. Распределение нагрузки по времени
def часы_пик(время):
    """Проверяет, попадает ли время в пиковые часы."""
    for начало, конец in ПИКОВЫЕ_ЧАСЫ_УТРОМ + ПИКОВЫЕ_ЧАСЫ_ВЕЧЕРОМ:
        if начало <= время.hour < конец:
            return True
    return False

# Добавляем нагрузку для каждого интервала
распределение_времени = pd.DataFrame({
    "Время": временные_интервалы,
    "Часы_Пик": [часы_пик(t) for t in временные_интервалы],
    "Загрузка": [0] * len(временные_интервалы)
})

# Расчет нагрузки в пиковые и непиковые часы
нагрузка_пик = int(КОЛИЧЕСТВО_АВТОБУСОВ * 0.7)  # Нагрузка в пиковые часы
нагрузка_непик = int(КОЛИЧЕСТВО_АВТОБУСОВ * 0.3)  # Нагрузка вне пиков

for i, row in распределение_времени.iterrows():
    распределение_времени.at[i, "Загрузка"] = нагрузка_пик if row["Часы_Пик"] else нагрузка_непик


In [6]:

# Шаг 3. Назначение автобусов и водителей
расписание = []

# Расписание водителей типа B (сутки работы, двое суток отдыха)
водители_B = max(1, (len(временные_интервалы) // (СМЕНА_B * 4)))  # Расчет минимального числа водителей B

for водитель_B in range(водители_B):
    начало = водитель_B * СМЕНА_B * 4
    for i in range(начало, начало + СМЕНА_B * 4, int(ВРЕМЯ_МАРШРУТА.total_seconds() // 60)):
        if i < len(временные_интервалы):
            расписание.append({
                "Тип_Водителя": "B",
                "ID_Водителя": f"B{водитель_B+1}",
                "ID_Автобуса": f"Автобус{(i % КОЛИЧЕСТВО_АВТОБУСОВ) + 1}",
                "Начало_Маршрута": распределение_времени["Время"].iloc[i],
                "Конец_Маршрута": распределение_времени["Время"].iloc[i] + ВРЕМЯ_МАРШРУТА
            })

# Расписание водителей типа A (8 часов с обедом)
водители_A = len(временные_интервалы) // (СМЕНА_A * 4)  # Расчет минимального числа водителей A

for водитель_A in range(водители_A):
    начало = водитель_A * СМЕНА_A * 4
    for i in range(начало, начало + СМЕНА_A * 4, int(ВРЕМЯ_МАРШРУТА.total_seconds() // 60)):
        if i < len(временные_интервалы):
            расписание.append({
                "Тип_Водителя": "A",
                "ID_Водителя": f"A{водитель_A+1}",
                "ID_Автобуса": f"Автобус{(i % КОЛИЧЕСТВО_АВТОБУСОВ) + 1}",
                "Начало_Маршрута": распределение_времени["Время"].iloc[i],
                "Конец_Маршрута": распределение_времени["Время"].iloc[i] + ВРЕМЯ_МАРШРУТА
            })

# Создание DataFrame для итогового расписания
итоговое_расписание = pd.DataFrame(расписание)
итоговое_расписание


,Тип_Водителя,ID_Водителя,ID_Автобуса,Начало_Маршрута,Конец_Маршрута
0,B,B1,Автобус1,1900-01-01 06:00:00,1900-01-01 07:15:00
1,B,B1,Автобус6,1900-01-02 00:45:00,1900-01-02 02:00:00
2,A,A1,Автобус1,1900-01-01 06:00:00,1900-01-01 07:15:00
3,A,A2,Автобус3,1900-01-01 14:00:00,1900-01-01 15:15:00
